# AdventureWorks Relational Postgres Lab

### Introduction

In this lesson, we will work with the adventureworks database in postgres.  Let's get started.

### Loading our data


We can begin by making sure our postgres application is running (if on a Mac, look for the elephant).  Once running we'll use the instructions in the [Adventureworks for postgres](https://github.com/lorint/AdventureWorks-for-Postgres) repo.

Move into the `install-script` directory.  Then run the following:

```bash
psql -c "CREATE DATABASE \"adventureworks\";"
psql -d adventureworks < install.sql
```

One thing confusing is that if we simply type `\dt` to display our tables, we won't find any listed.  This is because all of our tables are under different schemas.  We can see this if we first connect to the adventureworks database and run the following:

```sql
\c "Adventureworks"
\dt (humanresources|person|production|purchasing|sales).*
```

So as we can see there are indeed a lot of tables in our database.  And we can query any of those tables with the schema prefix like so.

```sql
select * from person.address limit 1;
```

The database is pretty complicated, and you can see all of the relations from the [erd link](https://i0.wp.com/improveandrepeat.com/wp-content/uploads/2018/12/AdvWorksOLTPSchemaVisio.png?ssl=1).  But we will stick to the tables in the sales schema.  Let's get started.



### Beginning our queries

We can connect to our database using the postgres library.

In [1]:
import warnings
warnings.filterwarnings('ignore')


In [4]:
import pandas as pd
import psycopg2

conn = psycopg2.connect(database="Adventureworks", user="postgres")

And from there, we can use pandas to read from our database.  We'll can begin with our customers table.

In [5]:
pd.read_sql("select * from sales.Customer limit 5", conn)

,customerid,personid,storeid,territoryid,rowguid,modifieddate
0,1,None,934,1,3f5ae95e-b87d-4aed-95b4-c3797afcb74f,2014-09-12 11:15:07.263
1,2,None,1028,1,e552f657-a9af-4a7d-a645-c429d6e02491,2014-09-12 11:15:07.263
2,3,None,642,4,130774b1-db21-4ef3-98c8-c104bcd6ed6d,2014-09-12 11:15:07.263
3,4,None,932,4,ff862851-1daa-4044-be7c-3e85583c054d,2014-09-12 11:15:07.263
4,5,None,1026,4,83905bdc-6f5e-4f71-b162-c98da069f38a,2014-09-12 11:15:07.263


And then let's view some of the data in our `SalesOrderHeader` table.

> Notice below we have a `.T` at the very end.  This is to `transpose` our dataframe -- which sometimes makes it easier to see all of the columns.  The `0` and `1` are to describe each of the rows of data.

In [44]:
pd.read_sql("select * from sales.SalesOrderHeader limit 2", conn).T

,0,1
salesorderid,43659,43660
revisionnumber,8,8
orderdate,2011-05-31 00:00:00,2011-05-31 00:00:00
duedate,2011-06-12 00:00:00,2011-06-12 00:00:00
shipdate,2011-06-07 00:00:00,2011-06-07 00:00:00
status,5,5
onlineorderflag,False,False
purchaseordernumber,PO522145787,PO18850127500
accountnumber,10-4020-000676,10-4020-000117
customerid,29825,29672


So we can see that our `SalesOrderHeader` table has a good amount of foreign keys including `customerId`.

Let's start by getting a sense of our data.  What is the earliest date of our orders.

In [20]:
pd.read_sql("select min(orderdate) from sales.SalesOrderHeader", conn).T

,0
min,2011-05-31


And what is the latest date of our orders.

In [21]:
pd.read_sql("select max(orderdate) from sales.SalesOrderHeader", conn).T

,0
max,2014-06-30


So we can see that our most recent orders were placed at the end June of 2014.  

Begin by finding the the top five total amounts spent by customers since July 1 2013.  Return both the customerIds and the total amounts spent.

In [24]:
query = """select customerId, sum(totaldue) as total_amount from sales.SalesOrderHeader 
where orderdate > '2013-07-01'::date
group by customerId
order by total_amount desc limit 5;"""
pd.read_sql(query, conn)

,customerid,total_amount
0,29641,419331.7837
1,29629,389437.6577
2,29923,386059.4979
3,29736,363872.0389
4,29701,355051.0477


Next let's find the names of the top five products that brought in the most amount of revenue, and the amount of revenue for each since July 1st 2013 -- we can use the `salesOrderDetail.modifiedDate` column as a proxy for the dates of each sale. 

In [26]:
query = """select production.Product.name, sum(unitprice) as total_price from sales.salesOrderDetail
join production.Product on production.Product.ProductID = sales.salesOrderDetail.ProductID
where sales.salesOrderDetail.modifiedDate > '2013-07-01'::date
 group by (production.Product.productid, production.Product.name) order by total_price desc limit 5;"""
pd.read_sql(query, conn)

,name,total_price
0,"Mountain-200 Black, 38",1.259927e+06
1,"Mountain-200 Black, 42",1.222931e+06
2,"Mountain-200 Silver, 38",1.156979e+06
3,"Mountain-200 Black, 46",1.141849e+06
4,"Mountain-200 Silver, 46",1.128350e+06


So now that we've done some analysis on the products, let's take a look at the countries.  Find the country region codes related to the countries that have had the most sales in the past year.

In [43]:
query = """select sales.SalesOrderHeader.territoryid, sum(totaldue) as total_amount 
from sales.SalesOrderHeader 
where orderdate > '2013-07-01'::date
group by sales.SalesOrderHeader.territoryId
order by total_amount desc
"""
pd.read_sql(query, conn)

,territoryid,total_amount
0,4,9.994438e+06
1,1,7.492804e+06
2,6,6.367257e+06
3,9,6.154403e+06
4,10,5.047151e+06
5,7,4.370710e+06
6,8,3.630621e+06
7,3,2.791293e+06
8,5,2.351479e+06
9,2,2.168895e+06


### Looking at Salespeople

Next, let's try to find the top performing salespeople.  To begin, let's take a look at our salesperson table.

In [40]:
pd.read_sql("select * from sales.SalesPerson limit 2;", conn).T

,0,1
businessentityid,274,275
territoryid,NaN,2.0
salesquota,NaN,300000.0
bonus,0.0,4100.0
commissionpct,0.0,0.012
salesytd,559697.5639,3763178.1787
saleslastyear,0.0,1750406.4785
rowguid,48754992-9ee0-4c0e-8c94-9451604e3e02,1e0a7274-3064-4f58-88ee-4c6586c87169
modifieddate,2010-12-28 00:00:00,2011-05-24 00:00:00


We can see that we have a `salesytd` column as well as a `modifieddate` column.  One item that may be difficult to identify is the sales person id.  This it turns out is the `businessentityid`.  

Now let's find the max salesperson for territory 4.

In [56]:
pd.read_sql("select * from sales.SalesPerson order by salesytd desc limit 1;", conn)

,businessentityid,territoryid,salesquota,bonus,commissionpct,salesytd,saleslastyear,rowguid,modifieddate
0,276,4,250000.0,2000.0,0.015,4.251369e+06,1.439156e+06,4dd9eee4-8e81-4f8c-af97-683394c1f7c0,2011-05-24


And from there, let's find all of the products that he sold.

In [67]:
pd.read_sql("select * from sales.salesOrderDetail limit 2;", conn) 

,salesorderid,salesorderdetailid,carriertrackingnumber,orderqty,productid,specialofferid,unitprice,unitpricediscount,rowguid,modifieddate
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,b207c96d-d9e6-402b-8470-2cc176c42283,2011-05-31
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,7abb600d-1e77-41be-9fe5-b9142cfc08fa,2011-05-31


Now let's the top three items that the salesperson with id 276 sold.  

> Some of the foreign keys may be tricky to figure out but use your best guess.

In [73]:
query = """select count(*) as amount, production.Product.name from sales.salesorderheader 
join sales.salesOrderDetail on sales.salesOrderDetail.salesorderid = sales.salesorderheader.salesorderid
join production.Product on production.Product.ProductID = sales.salesOrderDetail.ProductID
where salespersonid = 276 group by production.Product.name order by amount desc limit 3;"""
pd.read_sql(query, conn)

,amount,name
0,141,"Long-Sleeve Logo Jersey, L"
1,135,AWC Logo Cap
2,124,"Sport-100 Helmet, Blue"
